In [20]:
#import os
#import glob
import numpy as np
import torch
#from torchvision import models
from torch import nn, optim
import torch.nn.functional as F

#from tqdm import tqdm_notebook
from tqdm.notebook import tnrange

In [2]:
input = np.ones((3,28,28))


In [3]:
xray_classification = np.load('../data/npy_files/xray classification.npy', allow_pickle=True)
xray_classification = xray_classification.item()

In [4]:
pixel_classification = np.load('../data/npy_files/pixel classification.npy', allow_pickle=True)
pixel_classification = pixel_classification.item()

# construct datasets of samples

In [5]:
dataset_dic = {}
for key in ['domain 1', 'domain 2', 'domain 3']:
    x, y = xray_classification[key].shape
    dataset_dic[key] = []
    for i in range(x):
        for j in range(y):
            temp_vec = np.append(xray_classification[key][i][j], pixel_classification[key][i][j])
            dataset_dic[key].append(temp_vec)

In [6]:
x_d1 = np.array(dataset_dic['domain 1'])
y_d1 = np.ones(x_d1.shape[0])

x_d2 = np.array(dataset_dic['domain 2'])
y_d2 = np.ones(x_d2.shape[0]) * 2

x_d3 = np.array(dataset_dic['domain 3'])
y_d3 = np.ones(x_d3.shape[0]) * 3

In [7]:
data_x = np.vstack((x_d1, x_d2, x_d3))
data_y = np.hstack((y_d1, y_d2, y_d3))

In [8]:
#permutate the datasets 
indx = np.random.permutation(len(data_y))
data_x = data_x[indx]
data_y = data_y[indx]

In [9]:
training_size = 10000
testing_size = 1000

training_x = data_x[0:training_size]
training_y = data_y[0:training_size]

testing_x = data_x[training_size:training_size + testing_size]
testing_y = data_y[training_size:training_size + testing_size]

In [10]:
def extract_sample(n_way, n_support, n_query, datax, datay, test=False):
    """
    Cite: https://github.com/cnielly/prototypical-networks-omniglot
    """ 
    sample = []
    # K is a random ordering of the domains (1,2,3), chooses from unique values
    # of datay which is a number corresponding to the domain, n_way just says 
    # choose 3 from 1,2,3 without replacement
    K = np.random.choice(np.unique(datay), n_way, replace=False)
    for cls in K: # cls will == 1, 2, and 3 in some order
        datax_cls = datax[datay == cls] # Selects every data point associated with a given domain
        perm = np.random.permutation(datax_cls) # Randomizes
        sample_cls = perm[:(n_support+n_query)] # Selects a small set of the domain 
        sample.append(sample_cls) # Adds to the sample
    sample = np.array(sample).astype('float')
    sample = torch.from_numpy(sample).float()
    return ({
      'data_vector': sample,
      'n_way': n_way,
      'n_support': n_support,
      'n_query': n_query
      })

In [11]:
#define parameters for few-shot learning
n_way = 3
n_support = 10
n_query = 20
N = 3

In [12]:
sample = extract_sample(n_way, n_support, n_query, training_x, training_y)
sample['data_vector'].shape
# Sample is a dictionary containing the data vector (with the samples),
# the number of domains, the number of support and query samples
# So the data vector has 3 groups (each one corresponding to a domain) made up
# of 30 samples each, each sample is the sampled spectrum with the brightness 
# of the pixel appended to the very end
# And it is all a torch tensor

torch.Size([3, 30, 1971])

# model building

In [13]:
# constructing the embedding

def load_protonet_conv(x_dim, hid_dim, z_dim):

    """
    Cite: https://github.com/cnielly/prototypical-networks-omniglot
    
    Loads the prototypical network model
    Arg:
      x_dim (tuple): dimension of input data
      hid_dim (int): dimension of hidden layers
      z_dim (int): dimension of output
    Returns:
      Model (Class ProtoNet)
      """
    encoder = nn.Sequential(
                    nn.Linear(x_dim[-1], hid_dim),
                    nn.Linear(hid_dim, hid_dim),
                    nn.ReLU(),
                    nn.Linear(hid_dim, hid_dim),
                    nn.ReLU(),
                    nn.Linear(hid_dim, z_dim))

    return ProtoNet(encoder)

In [14]:
def euclidean_dist(x, y):
    """
    Cite: https://github.com/cnielly/prototypical-networks-omniglot
    Computes euclidean distance btw x and y
    Args:
      x (torch.Tensor): shape (n, d). n usually n_way*n_query
      y (torch.Tensor): shape (m, d). m usually n_way
    Returns:
      torch.Tensor: shape(n, m). For each query, the distances to each centroid
    """
    n = x.size(0)
    m = y.size(0)
    d = x.size(1)
    assert d == y.size(1)

    x = x.unsqueeze(1).expand(n, m, d)
    y = y.unsqueeze(0).expand(n, m, d)

    return torch.pow(x - y, 2).sum(2)

In [15]:
#Cite: https://github.com/cnielly/prototypical-networks-omniglot

class ProtoNet(nn.Module):
    def __init__(self, encoder):
        super(ProtoNet, self).__init__()
        self.encoder = encoder#.cuda()

    def set_forward_loss(self, sample):
        sample_vec = sample['data_vector']
        n_way, n_support, n_query = sample['n_way'], sample['n_support'], sample['n_query']
        x_support = sample_vec[:, :n_support]
        x_query = sample_vec[:, n_support:]
        
        target_inds = torch.arange(0, n_way).view(n_way, 1, 1).expand(n_way, n_query, 1).long()
        target_inds = torch.autograd.Variable(target_inds, requires_grad=False)
        target_inds = target_inds#.cuda()
        
        x_s = x_support.contiguous().view(n_way * n_support, *x_support.size()[2:])
        x_q = x_query.contiguous().view(n_way * n_query, *x_query.size()[2:])
        
        x = torch.cat([x_s,x_q], 0)
        z = self.encoder(x.float())
        z_dim = z.size(-1)
        
        z_proto = z[:n_way*n_support].view(n_way, n_support, z_dim).mean(1)
        z_query = z[n_way*n_support:]
        
        #compute distances
        dists = euclidean_dist(z_query, z_proto)

        #compute probabilities
        log_p_y = F.log_softmax(-dists, dim=1).view(n_way, n_query, -1)

        loss_val = -log_p_y.gather(2, target_inds).squeeze().view(-1).mean()
        _, y_hat = log_p_y.max(2)
        acc_val = torch.eq(y_hat, target_inds.squeeze()).float().mean()
        return loss_val, {
            'loss': loss_val.item(),
            'acc': acc_val.item(),
            'y_hat': y_hat
            }

# Training 

In [16]:
#training 
def train(model, optimizer, n_way, n_support, n_query, max_epoch, epoch_size):
    """
    Cite: https://github.com/cnielly/prototypical-networks-omniglot
    Trains the protonet
    Args:
      model
      optimizer
      n_way (int): number of classes in a classification task
      n_support (int): number of labeled examples per class in the support set
      n_query (int): number of labeled examples per class in the query set
      max_epoch (int): max epochs to train on
      epoch_size (int): episodes per epoch
    """
    #divide the learning rate by 2 at each epoch, as suggested in paper
    scheduler = optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.5, last_epoch=-1)
    epoch = 0 
    stop = False #status to know when to stop
    
    print("---------START training-------------")
    
    while epoch < max_epoch and not stop:
        running_loss = 0.0
        running_acc = 0.0
        for episode in tnrange(epoch_size, desc="Epoch {:d} train".format(epoch+1)):
            if episode% 100 == 0:
                print("=", end='')
            sample = extract_sample(n_way, n_support, n_query, training_x, training_y)
            optimizer.zero_grad()
            loss, output = model.set_forward_loss(sample)
            running_loss += output['loss']
            running_acc += output['acc']
            loss.backward()
            optimizer.step()
        epoch_loss = running_loss / epoch_size
        epoch_acc = running_acc / epoch_size
        epoch += 1
        print('Epoch {:d} -- Loss: {:.4f} Acc: {:.4f}'.format(epoch,epoch_loss, epoch_acc))
        scheduler.step()

In [22]:
x_dim = sample['data_vector'].shape
hid_dim = 60
z_dim = 40

model = load_protonet_conv(
    x_dim,
    hid_dim,
    z_dim)

optimizer = optim.Adam(model.parameters(), lr = 0.001)
max_epoch = 5
epoch_size = 1000

train(model, optimizer, n_way, n_support, n_query, max_epoch, epoch_size)

---------START training-------------


Epoch 1 train:   0%|          | 0/1000 [00:00<?, ?it/s]

==========Epoch 1 -- Loss: 0.2527 Acc: 0.8940


Epoch 2 train:   0%|          | 0/1000 [00:00<?, ?it/s]

==========Epoch 2 -- Loss: 0.1690 Acc: 0.9268


...

==========Epoch 3 -- Loss: 0.1501 Acc: 0.9351


Epoch 4 train:   0%|          | 0/1000 [00:00<?, ?it/s]

==========Epoch 4 -- Loss: 0.1394 Acc: 0.9413


Epoch 5 train:   0%|          | 0/1000 [00:00<?, ?it/s]

==========Epoch 5 -- Loss: 0.1309 Acc: 0.9444


In [23]:
def test(model, test_x, test_y, n_way, n_support, n_query, test_episode):
    """
    Cite: https://github.com/cnielly/prototypical-networks-omniglot
    """
    running_loss = 0.0
    running_acc = 0.0
    for episode in tnrange(test_episode):
        sample = extract_sample(n_way, n_support, n_query, testing_x, testing_y)
        loss, output = model.set_forward_loss(sample)
        running_loss += output['loss']
        running_acc += output['acc']
    avg_loss = running_loss / test_episode
    avg_acc = running_acc / test_episode
    print('Test results -- Loss: {:.4f} Acc: {:.4f}'.format(avg_loss, avg_acc))

n_way = 3
n_support = 5
n_query = 10

test_x = testing_x
test_y = testing_y

test_episode = 2000

test(model, test_x, test_y, n_way, n_support, n_query, test_episode)

  0%|          | 0/2000 [00:00<?, ?it/s]

Test results -- Loss: 0.2337 Acc: 0.9122


In [19]:
N = 100
acc_1 = 0
acc_2 = 0
acc_3 = 0
for _ in range(N):
    sample = extract_sample(n_way, n_support, 1000, training_x, training_y)
    loss, output = model.set_forward_loss(sample)
    output['y_hat'][0]
    acc_1 += np.count_nonzero(output['y_hat'][0] == 0)/1000
    acc_2 += np.count_nonzero(output['y_hat'][1] == 1)/1000
    acc_3 += np.count_nonzero(output['y_hat'][2] == 2)/1000
#region one classification accuracy
print("for region one", acc_1/N)
print("for region two", acc_2/N)
print("for region three", acc_3/N)

for region one 0.9499800000000002
for region two 0.9490099999999997
for region three 0.9440400000000001


### output

- n_way = 3, n_support = 5, n_query = 10: training accuracy = 0.9372; testing accuracy = 0.9012;
- n_way = 3, n_support = 5, n_query = 5: training accuracy = 0.9373; testing accuracy = 0.9011;
- n_way = 3, n_support = 5, n_query = 20: training accuracy = 0.949; testing accuracy = 0.897;
- n_way = 3, n_support = 20, n_query = 10: training accuracy = 0.9470; testing accuracy = 0.8967;
- n_way = 3, n_support = 10, n_query = 20: training accuracy = 0.9506; testing accuracy = 0.8917;

- n_way = 2, n_support = 5, n_query = 20: training accuracy = 0.9681; testing accuracy = 0.9051;
- n_way = 2, n_support = 5, n_query = 10: training accuracy = 0.9665; testing accuracy = 0.9015;
